In [15]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [16]:
misc_df = pd.read_csv("./data/misc_stats.csv")
opponent_per_100_df = pd.read_csv("./data/opponent_per_100_poss.csv")
opponent_shooting_df = pd.read_csv("./data/opponent_shooting.csv")
opponent_stats_df = pd.read_csv("./data/opponent_stats.csv")
opponents_per_game_df = pd.read_csv("./data/opponents_per_game.csv")
team_per_game_df = pd.read_csv("./data/team_per_game.csv")
team_shooting_df = pd.read_csv("./data/team_shooting.csv")
team_stats_df = pd.read_csv("./data/team_stats.csv")

In [17]:
team_stats_df

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,Dallas Mavericks,67,16180,2787,6027,0.462,1026,2779,0.369,...,0.773,712,2438,3150,1640,419,334,857,1276,7796
1,2.0,Milwaukee Bucks*,65,15650,2830,5928,0.477,893,2510,0.356,...,0.742,617,2745,3362,1685,480,390,969,1247,7712
2,3.0,Houston Rockets,64,15435,2632,5803,0.454,986,2832,0.348,...,0.787,663,2212,2875,1375,541,329,939,1382,7560
3,4.0,Portland Trail Blazers,66,15890,2763,5998,0.461,829,2228,0.372,...,0.798,666,2335,3001,1333,401,411,858,1412,7500
4,5.0,Atlanta Hawks,67,16280,2723,6067,0.449,805,2416,0.333,...,0.790,661,2237,2898,1605,523,341,1086,1548,7488
5,6.0,New Orleans Pelicans,64,15510,2729,5902,0.462,896,2409,0.372,...,0.729,719,2290,3009,1728,486,328,1038,1345,7436
6,7.0,Los Angeles Clippers,64,15435,2664,5739,0.464,778,2123,0.366,...,0.792,701,2368,3069,1526,456,318,946,1411,7436
7,8.0,Washington Wizards,64,15435,2682,5821,0.461,788,2117,0.372,...,0.787,644,2024,2668,1621,519,273,900,1445,7399
8,9.0,Memphis Grizzlies,65,15625,2779,5913,0.470,710,2019,0.352,...,0.761,679,2357,3036,1757,520,363,992,1354,7321
9,10.0,Phoenix Suns,65,15675,2650,5709,0.464,727,2061,0.353,...,0.826,637,2163,2800,1766,507,261,980,1439,7316
